# Logging and tests utilities

> Utilities to ease the setting of logging and tests

In [ ]:
#| default_exp lgtst

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import logging
from logging.handlers import RotatingFileHandler
from pathlib import Path

from hopsa import ossys

## Logging

#### Logging Levels

Python logging has five standard levels, in increasing order of severity:

- DEBUG (10): Detailed information, typically useful for diagnosing problems
- INFO (20): Confirmation that things are working as expected
- WARNING (30): An indication something unexpected happened, but the program still works
- ERROR (40): Due to a more serious problem, the software couldn't perform some function
- CRITICAL (50): A very serious error, indicating the program may be unable to continue

The function does return the root Logger, though typically you would not use it directly.

In [ ]:
#| export
def set_logging(
        level: int = logging.INFO, # The logging level
        format_file: str = '%(asctime)s - %(name)s - %(levelname)s - %(message)s', # The logging format for the file
        format_console: str = '%(levelname)s - %(message)s', # The logging format for the console
        datefmt: str = '%Y-%m-%d %H:%M:%S', # The date format
        log_dir: str = None, # The logging directory, if None, logs to console
        filemode: str = 'a', # The logging file mode. 'a' for append, 'w' for overwrite
        backupCount: int = 5, # The number of backup files to keep
        maxBytes: int = 1024*1024*5, # The maximum size of the log file in bytes
    ) -> logging.Logger:
    """Set up the root Logger"""

    if log_dir is None:
        log_dir = ossys.get_project_root() / 'logs'
    else:
        log_dir = Path(log_dir)
    
    log_dir.mkdir(exist_ok=True, parents=True)

    log_file = log_dir / f'{ossys.get_project_name()}.log'

    # Configure root logger
    root_logger = logging.getLogger()
    root_logger.setLevel(level)

    # Create console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(level)
    console_handler.setFormatter(logging.Formatter(format_console))

    # Create rotating file handler
    file_handler = RotatingFileHandler(
        log_file,
        maxBytes=maxBytes,
        backupCount=backupCount,
        encoding='utf-8'
        )
    file_handler.setLevel(level)
    file_handler.setFormatter(logging.Formatter(format_file, datefmt=datefmt))

    # Add handlers to root logger
    root_logger.addHandler(console_handler)
    root_logger.addHandler(file_handler)

    # Log startup information
    logging.info(f"Log file: {log_file}")
    logging.info(f"Log file mode: {filemode}")
    logging.info(f"Log backup count: {backupCount}")
    logging.info(f"Log max bytes: {maxBytes}")
    
    return root_logger

#### Example usage

We must make sure the logging works when we are testing indivudual notebooks and when running the complete code. To accomplish this we can do the following.

1. First in `run.py` or other entry point of the project, add the following to get the logging when the complete code is run:

>```python
>from hopsa import set_logging
>
>if __name__ == "__main__":
>    log_dir = "../logs"
>    set_logging(log_dir=log_dir, level=10)
>```

or

>```python
>import logging
>from hopsa import set_logging
>
>if __name__ == "__main__":
>    log_dir = "../logs"
>    set_logging(log_dir=log_dir, level=logging.DEBUG)
>```

2. Then in each module/notebook, you create module-specific loggers:

At the top of each notebook (00_core.ipynb, 02_features.ipynb, etc.)

>```python
>#| export
>import logging
>```

>```python
>#| eval: false
>from hopsa import lgtst
>```

>```python
>#| eval: false
>lgtst.set_logging(log_dir="../logs", level=logging.DEBUG)
>```

>```python
>#| export
>logger = logging.getLogger(__name__)
>```

The `#| export` nbdev directive makes sure this cell will be used in the python module.
The `#| eval: false` nbdev directive makes sure this cell will run when we run the notebook, but it won't be used in the Python module and it also won't be tested when we run `nbdev_prepare`.

Then use the logger throughout the module

>```python
>logger.debug("Debug message")
>logger.info("Info message")
>```

In [ ]:
rt_logger = set_logging()

INFO - Log file: /home/jelle/code/hopsa/logs/hopsa.log
INFO - Log file: /home/jelle/code/hopsa/logs/hopsa.log
INFO - Log file mode: a
INFO - Log file mode: a
INFO - Log backup count: 5
INFO - Log backup count: 5
INFO - Log max bytes: 5242880
INFO - Log max bytes: 5242880


In [ ]:
for h in rt_logger.handlers:
    h.close()
    rt_logger.removeHandler(h)
    print(f"Removed handler: {h}")

Removed handler: <StreamHandler stderr (INFO)>
Removed handler: <RotatingFileHandler /home/jelle/code/hopsa/logs/hopsa.log (INFO)>


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()